In [1]:
from transformer import Transformer
import silence_tensorflow.auto
import numpy as np

t = Transformer(
    num_layers=3,
    d_model=12,
    num_heads=3,
    dff=24,
    input_dimensions=(4,5,24),
    target_dimensions=(4,2),
    rate=0.1,
)

i = np.random.random((10, 4,5,24))
o = np.random.random((10, 4,2))

inputs = [i, o]



TypeError: random() takes at most 1 positional argument (4 given)